In [46]:
from pathlib import Path
from typing import List, Any, Dict
import requests
import pandas as pd

In [12]:
r = requests.get('https://api.beta.ons.gov.uk/v1/datasets')
dataset_titles = []
dataset_descriptions: List[Any] = []
dataset_ids: List[Any] = []
for i in range(len(r.json()['items'])):
    dataset_titles.append(r.json()['items'][i]['title'])
    dataset_descriptions.append(r.json()['items'][i]['description'])
    dataset_ids.append(r.json()['items'][i]['id'])

table_of_codes = pd.DataFrame(data={'title': dataset_titles,
                   'description': dataset_descriptions,
                  'dataset_id': dataset_ids})

## find information about a dataset
we need to know the following items to make a correct api call:
* edition
* version
* time
* aggregate 
* geography
* dimension

> This allows querying of a single observation/value by providing one option per dimension, but will also allow one of these dimensions to be a ‘wildcard’ and return all values for this dimension.

`/datasets/{datasetId}/editions/{edition}/versions/{version}/observations?time={timeLabel}&geography={geographyID}&dimension3={dimension3ID}&dimension4={dimension4ID}...`

In [13]:
def post(extension, is_ok=False):
    r = requests.get(f'https://api.beta.ons.gov.uk/v1/' + extension)
    if is_ok:
        print(r)
    return r.json()

`/datasets/cpih01/editions/time-series/versions/6/observations?time=Oct-11&geography=K02000001&aggregate=cpih1dim1A0`

### editions param

In [14]:
editions = post('/datasets/cpih01/editions')

In [15]:
editions['items'][0]['edition']

'time-series'

In [16]:
mother_df = pd.DataFrame.from_dict(post('/datasets/cpih01/editions/time-series/versions')['items'][3], orient='index').T

In [17]:
for i in range(len(post('/datasets/cpih01/editions/time-series/versions')['items'])):
    try:
        my_dict = post('/datasets/cpih01/editions/time-series/versions')['items'][i+1]
        df = pd.DataFrame.from_dict(my_dict, orient='index').T
        mother_df = mother_df.append(df)
    except:
        pass


In [18]:
mother_df.head()

,collection_id,dimensions,downloads,edition,id,links,release_date,state,version
0,12345,[{'description': 'Includes the complete availa...,{'csv': {'href': 'https://download.beta.ons.go...,time-series,f75c9f72-ae23-4271-a9f2-8ccca43b5b38,{'dataset': {'href': 'https://api.beta.ons.gov...,2018-02-13T00:00:00.000Z,published,4
0,12345,[{'description': 'Includes the complete availa...,{'csv': {'href': 'https://download.beta.ons.go...,time-series,bede15c1-52b5-4bb8-be6a-67fd60ef837f,{'dataset': {'href': 'https://api.beta.ons.gov...,2017-12-12T00:00:00.000Z,published,2
0,12345,[{'description': 'Includes the complete availa...,{'csv': {'href': 'https://download.beta.ons.go...,time-series,1cde4c8c-8dd3-49a3-8641-91e9309eded0,{'dataset': {'href': 'https://api.beta.ons.gov...,2018-01-16T00:00:00.000Z,published,3
0,12345,[{'description': 'Includes the complete availa...,{'csv': {'href': 'https://download.beta.ons.go...,time-series,f75c9f72-ae23-4271-a9f2-8ccca43b5b38,{'dataset': {'href': 'https://api.beta.ons.gov...,2018-02-13T00:00:00.000Z,published,4


* find out how many versions there are
* for each version, grab all the dimensions
* select a trivial day in the time slot so we can see all the dimensions

In [19]:
post('/code-lists/mid-year-pop-geography/editions', is_ok=True)

<Response [200]>


{'count': 1,
 'items': [{'edition': 'one-off',
   'label': 'geography',
   'links': {'codes': {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mid-year-pop-geography/editions/one-off/codes'},
    'editions': {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mid-year-pop-geography/editions'},
    'self': {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mid-year-pop-geography/editions/one-off',
     'id': 'one-off'}}}],
 'limit': 1,
 'offset': 0,
 'total_count': 1}

In [20]:
post('/datasets/cpih01/editions/time-series/versions/6')['dimensions']

[{'description': 'Includes the complete available monthly time series for CPIH. Yearly and quarterly data are also available from the main ONS website.',
  'href': 'https://api.beta.ons.gov.uk/v1/code-lists/time',
  'id': 'time',
  'links': {'code_list': {}, 'options': {}, 'version': {}},
  'name': 'time'},
 {'description': 'Provides data at national level for the United Kingdom only. No breakdown of this data to smaller areas is available.',
  'href': 'https://api.beta.ons.gov.uk/v1/code-lists/uk-only',
  'id': 'uk-only',
  'links': {'code_list': {}, 'options': {}, 'version': {}},
  'name': 'geography'},
 {'description': 'The CPIH indices are categorised according to the international classification system for household consumption expenditures known as COICOP (Classification Of Individual Consumption According to Purpose). COICOP is a hierarchical classification system comprising of Divisions (COICOP2) e.g. 01 Food & non-alcoholic beverages, Groups (COICOP3) e.g. 01.1 Food, and Class

`observations?time={timeLabel}&geography={geographyID}&dimension3={dimension3ID}&dimension4={dimension4ID}`

In [21]:
post('/datasets/cpih01/editions/time-series/versions/30/observations?time=Oct-11&geography=K02000001&aggregate=*').keys()

dict_keys(['@context', 'dimensions', 'limit', 'links', 'observations', 'offset', 'total_observations', 'unit_of_measure', 'usage_notes'])

In [22]:
post('/datasets/cpih01/editions/time-series/versions/30/observations?time=Jan-03&geography=K02000001&aggregate=*')\
    ['observations'][0]['dimensions']['aggregate']['label']

'09.1.2 Photographic, cinematographic and optical equipment'

In [23]:
for dimension in post('/datasets/cpih01/editions/time-series/versions/30/observations?time=Jan-03&geography=K02000001&aggregate=*')\
        ['observations'][0]['dimensions'].keys():
    print(dimension)

aggregate


Create a chart from Population Estimates for UK, Wales, etc.

* dataset
* edition
* version
* time
* aggregate
* geography
* dimension

code: mid-year-pop-est

In [35]:
dataset_id = 'mid-year-pop-est'

In [25]:
uk_population_dataset = post(f'/datasets/{dataset_id}', is_ok=True)

<Response [200]>


In [26]:
list(uk_population_dataset.keys())

['@context',
 'contacts',
 'description',
 'id',
 'keywords',
 'license',
 'links',
 'methodologies',
 'national_statistic',
 'next_release',
 'publications',
 'publisher',
 'qmi',
 'related_datasets',
 'release_frequency',
 'state',
 'theme',
 'title',
 'unit_of_measure']

In [27]:
uk_population_dataset

{'@context': 'https://cdn.ons.gov.uk/assets/json-ld/context.json',
 'contacts': [{'email': 'pop.info@ons.gov.uk ',
   'name': 'Neil Park ',
   'telephone': '+44 (0)1329 444661'}],
 'description': 'Estimates of the usual resident population for the UK as at 30 June of the reference year. Provided by administrative area, single year of age and sex.',
 'id': 'mid-year-pop-est',
 'keywords': ['Population'],
 'license': 'Open Government Licence v3.0',
 'links': {'access_rights': {},
  'editions': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/mid-year-pop-est/editions'},
  'latest_version': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/mid-year-pop-est/editions/time-series/versions/4',
   'id': '4'},
  'self': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/mid-year-pop-est'}},
 'methodologies': [{'description': 'Background information and methods changes for this year',
   'href': 'https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/metho

In [28]:
uk_population_dataset.keys()

dict_keys(['@context', 'contacts', 'description', 'id', 'keywords', 'license', 'links', 'methodologies', 'national_statistic', 'next_release', 'publications', 'publisher', 'qmi', 'related_datasets', 'release_frequency', 'state', 'theme', 'title', 'unit_of_measure'])

## fetch the latest version of this dataset

In [29]:
uk_pop_latest_version = uk_population_dataset['links']['latest_version']['id']
uk_pop_latest_version


'4'

In [30]:
uk_population_editions = post(f'/datasets/{dataset_id}/editions/time-series/versions/{uk_pop_latest_version}', is_ok=True)
uk_population_editions


<Response [200]>


{'@context': 'https://cdn.ons.gov.uk/assets/json-ld/context.json',
 'alerts': [],
 'collection_id': 'midyearpopest-7e05d4510fcc86e5182e9838ee4419a92e1e28e2d71885e5d1b1b10a11a33fd7',
 'dimensions': [{'href': 'https://api.beta.ons.gov.uk/v1/code-lists/calendar-years',
   'id': 'calendar-years',
   'links': {'code_list': {}, 'options': {}, 'version': {}},
   'name': 'time'},
  {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mid-year-pop-geography',
   'id': 'mid-year-pop-geography',
   'label': 'Geography',
   'links': {'code_list': {}, 'options': {}, 'version': {}},
   'name': 'geography'},
  {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mid-year-pop-sex',
   'id': 'mid-year-pop-sex',
   'links': {'code_list': {}, 'options': {}, 'version': {}},
   'name': 'sex'},
  {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mid-year-pop-age',
   'id': 'mid-year-pop-age',
   'links': {'code_list': {}, 'options': {}, 'version': {}},
   'name': 'age'}],
 'downloads': {'csv': {'href': 'htt

##### fetch the dimensions that this dataset has to build the rest of the api.
##### We use the code list for this, a separate service

In [37]:
uk_population_code_list = post(f'/code-lists/{dataset_id}', is_ok=True)

<Response [404]>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [38]:
dimensions = post(f'/datasets/{dataset_id}/editions/time-series/versions/{uk_pop_latest_version}/dimensions', is_ok=True)

<Response [200]>


In [39]:
dimensions['items'][3]
# 'time'
# 'sex'
# 'geography'
# 'age'

{'links': {'code_list': {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mid-year-pop-age',
   'id': 'mid-year-pop-age'},
  'options': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/mid-year-pop-est/editions/time-series/versions/4/dimensions/age/options',
   'id': 'age'},
  'version': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/mid-year-pop-est/editions/time-series/versions/4'}},
 'name': 'age'}

In [61]:
# check options for each dimension
dimension = 'sex'
options: Dict = post(f'/datasets/{dataset_id}/editions/time-series/versions/{uk_pop_latest_version}/dimensions/{dimension}/options', is_ok=True)
options['items']

<Response [200]>


[{'dimension': 'sex',
  'label': 'All',
  'links': {'code': {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mid-year-pop-sex/codes/0',
    'id': '0'},
   'code_list': {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mid-year-pop-sex',
    'id': 'mid-year-pop-sex'},
   'version': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/mid-year-pop-est/editions/time-series/versions/4',
    'id': '4'}},
  'option': '0'},
 {'dimension': 'sex',
  'label': 'Male',
  'links': {'code': {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mid-year-pop-sex/codes/1',
    'id': '1'},
   'code_list': {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mid-year-pop-sex',
    'id': 'mid-year-pop-sex'},
   'version': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/mid-year-pop-est/editions/time-series/versions/4',
    'id': '4'}},
  'option': '1'},
 {'dimension': 'sex',
  'label': 'Female',
  'links': {'code': {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mid-year-pop-sex/codes/2',
    'id': '

[{'dimension': 'age',
  'label': '0',
  'links': {'code': {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mid-year-pop-age/codes/0',
    'id': '0'},
   'code_list': {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mid-year-pop-age',
    'id': 'mid-year-pop-age'},
   'version': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/mid-year-pop-est/editions/time-series/versions/4',
    'id': '4'}},
  'option': '0'},
 {'dimension': 'age',
  'label': '1',
  'links': {'code': {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mid-year-pop-age/codes/1',
    'id': '1'},
   'code_list': {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mid-year-pop-age',
    'id': 'mid-year-pop-age'},
   'version': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/mid-year-pop-est/editions/time-series/versions/4',
    'id': '4'}},
  'option': '1'},
 {'dimension': 'age',
  'label': '10',
  'links': {'code': {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mid-year-pop-age/codes/10',
    'id': '10'},
  

In [75]:
# age = type in int for age, up to '90+'. Use wildcard if possible. e.g '21'
# time = id is the year e.g '2010'
# geography = geography codes. UK = K02000001
# sex = 'All': 0, 'Male': 1, 'Female': 2
# %%

post(f'/datasets/{dataset_id}'
     f'/editions/time-series'
     f'/versions/4/'
     f'observations?'
     f'time=2017&'
     f'geography=K02000001&'
     f'sex=0&'
     f'age=*', is_ok=True)

<Response [200]>


{'@context': 'https://cdn.ons.gov.uk/assets/json-ld/context.json',
 'dimensions': {'geography': {'option': {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mid-year-pop-geography/codes/K02000001',
    'id': 'K02000001'}},
  'sex': {'option': {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mid-year-pop-sex/codes/0',
    'id': '0'}},
  'time': {'option': {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/calendar-years/codes/2017',
    'id': '2017'}}},
 'limit': 10000,
 'links': {'dataset_metadata': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/mid-year-pop-est/editions/time-series/versions/4/metadata'},
  'self': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/mid-year-pop-est/editions/time-series/versions/4/observations?time=2017&geography=K02000001&sex=0&age=*'},
  'version': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/mid-year-pop-est/editions/time-series/versions/4',
   'id': '4'}},
 'observations': [{'dimensions': {'age': {'href': 'https://api.beta.ons.gov.uk/v1

{'@context': 'https://cdn.ons.gov.uk/assets/json-ld/context.json',
 'dimensions': {'aggregate': {'option': {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/cpih1dim1aggid/codes/cpih1dim1A0',
    'id': 'cpih1dim1A0'}},
  'geography': {'option': {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/uk-only/codes/K02000001',
    'id': 'K02000001'}}},
 'limit': 10000,
 'links': {'dataset_metadata': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/cpih01/editions/time-series/versions/5/metadata'},
  'self': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/cpih01/editions/time-series/versions/5/observations?time=*&aggregate=cpih1dim1A0&geography=K02000001'},
  'version': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/cpih01/editions/time-series/versions/5',
   'id': '5'}},
 'observations': [{'dimensions': {'time': {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/time/codes/Month',
     'id': 'Month',
     'label': 'Sep-15'}},
   'observation': '100.2'},
  {'dimensions': {'time': {'h